## **Chicago Type of Crime - Data Extraction**

**Labels:**
* Primary Type of Crime (column in dataset)

**Features:**

* Hour (possibly using one-hot vector)
* Day of Week (python datetime can compute day of week; possibly use one-hot vector)
* Holidays (binary flag)
Could find this online (e.g. https://gist.github.com/shivaas/4758439)
May need to verify sources
* Weather / Season (Daily, possibly hourly)
https://www.ncdc.noaa.gov/cdo-web/datasets
* Block +
* Local government spending  + taxes
* Income  & Population
* Link Separate Doc (census?) +
* Location description +
* District +
* Ward +
* Community Area +
* Latitude +
* Longitude +


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
# Ensure you click "Add Shortcut to My Drive" for the "IE 534 PROJECT" folder
df = pd.read_csv('/content/drive/MyDrive/IE 534 PROJECT/data/Crime 2021-2023.csv')

print(df.shape)


(449958, 30)


In [3]:
print(df.columns)

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards', 'Boundaries - ZIP Codes',
       'Police Districts', 'Police Beats'],
      dtype='object')


In [4]:
# Convert datetimes to pandas timestamp
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
df['Updated On'] = pd.to_datetime(df['Updated On'], format='%m/%d/%Y %I:%M:%S %p')

# Verify the conversion
print(df['Date'].dtype)  # Should output: datetime64[ns]
print(df['Updated On'].dtype)  # Should output: datetime64[ns]
# Date, Updated On

datetime64[ns]
datetime64[ns]


### **Get the holiday data from kaggle holiday dataset**

[kaggle US Holiday data](https://www.kaggle.com/datasets/jeremygerdes/us-federal-pay-and-leave-holidays-2004-to-2100-csv?resource=download)


In [5]:
import pandas as pd
# holiday dataframe
h_orig_df = pd.read_csv('/content/drive/MyDrive/IE 534 PROJECT/data/US_Holidays_Dataset.csv')

print(h_orig_df.shape)

(146100, 28)


In [6]:
# filter the dataframe and keep only those that are holiday
h_df = h_orig_df[['IS_HOLIDAY', 'A_DATE']].fillna(0)

h_df = h_df[h_df['IS_HOLIDAY'] == 1]
h_df['A_DATE'] = pd.to_datetime(h_df['A_DATE'], format = '%m/%d/%Y')

holidays_df = h_df[(h_df['A_DATE'].dt.year >= 2021) & (h_df['A_DATE'].dt.year <= 2023)]

print(holidays_df.shape)
print(holidays_df.head())

(33, 2)
        IS_HOLIDAY     A_DATE
109024         1.0 2021-01-01
109041         1.0 2021-01-18
109069         1.0 2021-02-15
109174         1.0 2021-05-31
109193         1.0 2021-06-19


In [7]:
# Initialize the Holidays column(binary flag) in the df
# df['Holidays'] = df['Date'].apply(lambda x: holidays_df) # not done yet

In [8]:
specific_date = df.loc[22650]['Date']
print(specific_date)
# df['Date']
# df['Updated On'].head()
exists = specific_date in holidays_df['A_DATE'].values
print(exists)

2021-02-15 08:30:00
False


**Import other datasets**

In [9]:
# Income and population for the chicago community areas (CCAs)
inc_df = pd.read_csv('/content/drive/MyDrive/IE 534 PROJECT/data/Per_Capita_Income.csv')
print(inc_df.shape)
print(inc_df.columns)

# Chicago Public Parks dataset
parks_df = pd.read_csv('/content/drive/MyDrive/IE 534 PROJECT/data/CPD_Parks_20241027.csv')
print(parks_df.shape)

# Weather dataset (2021-2023)
w_df = pd.read_csv('/content/drive/MyDrive/IE 534 PROJECT/data/2021-2023_weather_chicago.csv')
print(w_df.shape)
print(w_df.columns)
w_df['DATE'] = pd.to_datetime(w_df['DATE'], format = '%Y-%m-%d')


(78, 9)
Index(['Community Area Number', 'COMMUNITY AREA NAME',
       'PERCENT OF HOUSING CROWDED', 'PERCENT HOUSEHOLDS BELOW POVERTY',
       'PERCENT AGED 16+ UNEMPLOYED',
       'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA',
       'PERCENT AGED UNDER 18 OR OVER 64', 'PER CAPITA INCOME ',
       'HARDSHIP INDEX'],
      dtype='object')
(617, 81)
(138850, 22)
Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'PRCP', 'PSUN', 'SNOW', 'TAVG', 'TOBS', 'WT01', 'WT02', 'WT03', 'WT04',
       'WT05', 'WT06', 'WT08', 'WT09', 'WT10', 'WT11'],
      dtype='object')


**Merging Datasets for Crime, Holidays, and Weather**


In [10]:
# Install gdown if not already installed
!pip install gdown

import gdown

crime_url = 'https://drive.google.com/uc?id=1rQJDOjTWjYZRazb-Bv7v8K2TrYAgyLB-'
holiday_url = 'https://drive.google.com/uc?id=1ZcO_656lQpL0IwOzxmEfj-Vx1iky7JNk'


gdown.download(crime_url, 'crime.csv', quiet=False)
gdown.download(holiday_url, 'holidays.csv', quiet=False)

holidays_df = pd.read_csv('holidays.csv')  # holidays dataset
crime_df = pd.read_csv('crime.csv')        # crime dataset


# Convert A_DATE in holidays dataset to datetime format
holidays_df['A_DATE'] = pd.to_datetime(holidays_df['A_DATE'], errors='coerce')

crime_df['Date'] = pd.to_datetime(crime_df['Date'], errors='coerce')

crime_df['Date'] = crime_df['Date'].dt.normalize()

crime_df = crime_df.dropna(subset=['Date'])
holidays_df = holidays_df.dropna(subset=['A_DATE'])

print("Crime Date type:", crime_df['Date'].dtype)
print("Holiday A_DATE type:", holidays_df['A_DATE'].dtype)

merged_df = pd.merge(crime_df, holidays_df, left_on='Date', right_on='A_DATE', how='left')

merged_df.to_csv('merged_crime_holidays.csv', index=False)

import pandas as pd

# Load the datasets
crime_df = pd.read_csv('crime.csv')
holidays_df = pd.read_csv('holidays.csv')
merged_df = pd.read_csv('merged_crime_holidays.csv')

# Get the shape of the dataframes (rows and columns)
crime_shape = crime_df.shape
holidays_shape = holidays_df.shape
merged_shape = merged_df.shape

print(f"Crime dataset: {crime_shape[0]} rows, {crime_shape[1]} columns")
print(f"Holidays dataset: {holidays_shape[0]} rows, {holidays_shape[1]} columns")
print(f"Merged dataset: {merged_shape[0]} rows, {merged_shape[1]} columns")


Downloading...
From (original): https://drive.google.com/uc?id=1rQJDOjTWjYZRazb-Bv7v8K2TrYAgyLB-
From (redirected): https://drive.google.com/uc?id=1rQJDOjTWjYZRazb-Bv7v8K2TrYAgyLB-&confirm=t&uuid=9063084e-8683-4ad9-b04a-0449580f87af
To: /content/crime.csv
100%|██████████| 120M/120M [00:00<00:00, 208MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZcO_656lQpL0IwOzxmEfj-Vx1iky7JNk
To: /content/holidays.csv
100%|██████████| 27.9M/27.9M [00:00<00:00, 157MB/s] 
<ipython-input-10-b3707b6c93b3>:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crime_df['Date'] = pd.to_datetime(crime_df['Date'], errors='coerce')


Crime Date type: datetime64[ns]
Holiday A_DATE type: datetime64[ns]
Crime dataset: 449958 rows, 30 columns
Holidays dataset: 146100 rows, 28 columns
Merged dataset: 449958 rows, 58 columns


In [11]:
import pandas as pd
import gdown

# Step 1: Download the weather dataset
weather_url = 'https://drive.google.com/uc?id=1hJQ8cDKQIDmVuqB2M0ccyGChJgq5W3o9'
gdown.download(weather_url, 'weather.csv', quiet=False)

# Step 2: Load the weather dataset with error handling
try:
    weather_df = pd.read_csv('weather.csv', encoding='utf-8', on_bad_lines='skip', delimiter=',')
except pd.errors.ParserError:
    print("Error parsing the CSV file. Please check the file format.")

merged_crime_holiday_df = pd.read_csv('merged_crime_holidays.csv')

weather_df['DATE'] = pd.to_datetime(weather_df['DATE'], errors='coerce')
weather_df['DATE'] = weather_df['DATE'].dt.normalize()

weather_df = weather_df.dropna(subset=['DATE'])

weather_df = weather_df.drop_duplicates(subset=['DATE'])

merged_crime_holiday_df['Date'] = pd.to_datetime(merged_crime_holiday_df['Date'], errors='coerce')
merged_crime_holiday_df['Date'] = merged_crime_holiday_df['Date'].dt.normalize()

merged_df = pd.merge(merged_crime_holiday_df, weather_df, left_on='Date', right_on='DATE', how='left')

merged_df.to_csv('final_merged_crime_holiday_weather.csv', index=False)

crime_holiday_shape = merged_crime_holiday_df.shape
weather_shape = weather_df.shape
merged_df_shape = merged_df.shape

print(f"Crime-Holiday dataset: {crime_holiday_shape[0]} rows, {crime_holiday_shape[1]} columns")
print(f"Weather dataset: {weather_shape[0]} rows, {weather_shape[1]} columns")
print(f"Final merged dataset: {merged_df_shape[0]} rows, {merged_df_shape[1]} columns")


Downloading...
From: https://drive.google.com/uc?id=1hJQ8cDKQIDmVuqB2M0ccyGChJgq5W3o9
To: /content/weather.csv
100%|██████████| 13.7M/13.7M [00:00<00:00, 139MB/s]


Crime-Holiday dataset: 449958 rows, 58 columns
Weather dataset: 1095 rows, 22 columns
Final merged dataset: 449958 rows, 80 columns


### **Separate Debugging and Working datasets**

In [12]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(merged_df, test_size= 0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)
debug_df = merged_df.sample(n=1000, random_state=42)

Verify debugging size

In [13]:
print(debug_df.shape)

(1000, 80)


Pickle Data

In [14]:

merged_df.to_pickle('final_merged_crime_holiday_weather.pkl')

In [15]:
loaded_df = pd.read_pickle('final_merged_crime_holiday_weather.pkl')

In [16]:
print(loaded_df.shape)

(449958, 80)


In [17]:
loaded_df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT10,WT11
0,13211146,JG424812,2021-01-01,035XX W DIVERSEY AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13204236,JG416567,2021-01-01,0000X E 61ST ST,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,RESIDENCE,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13218115,JG431987,2021-01-01,052XX W MONROE ST,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,RESIDENCE,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12366492,JE231493,2021-01-01,014XX N KOLIN AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,APARTMENT,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12416112,JE292434,2021-01-01,002XX W HURON ST,0281,CRIMINAL SEXUAL ASSAULT,NON-AGGRAVATED,HOSPITAL BUILDING / GROUNDS,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
